In [1]:
import numpy as np
import math
# %timeit -n10 

### Fastest nth root

In [2]:
x = np.random.random((500, 500))
y = np.random.random((500, 500))

In [3]:
%timeit -n100 x**(1/40)

4.2 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit -n100 np.power(x, 1/40)

4.21 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest random float [0, 1[

In [5]:
from random import random as fastRandom
%timeit -n40000 fastRandom()

51 ns ± 5.1 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


In [6]:
%timeit -n40000 np.random.random()

348 ns ± 40.3 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


### Fastest power

In [7]:
%timeit -n100 x ** y
%timeit -n100 np.power(x, y)

4.28 ms ± 74.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.28 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit -n100 x ** 2
%timeit -n100 np.power(x, 2)

103 µs ± 17.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.33 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### ==> Numpy.power is faster for small powers
#### ==> Python ** is faster for large powers

### Fastest Sqrt

In [9]:
%timeit -n500 x ** (1/2)
%timeit -n500 np.sqrt(x)

203 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
196 µs ± 9.26 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


### Fastest Descending sorting in numpy

In [10]:
a = np.random.rand(40,40,80)

In [11]:
%timeit -n100 np.flip(np.sort(a, axis=-1, kind='quicksort'), axis=-1)
%timeit -n100 np.sort(a, axis=-1, kind='quicksort')[::-1]
%timeit -n100 (-np.sort(-a, axis=-1, kind='quicksort'))

2.7 ms ± 86.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.77 ms ± 92.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.85 ms ± 82.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest derangment
i.e. random shuffling such that no element is at the same place as in the begining, i.e. without fixed point

In [12]:
def random_derangement_1(n):
    while True:
        v = np.arange(n)
        for j in np.arange(n - 1, -1, -1):
            p = np.random.randint(0, j+1)
            if v[p] == j:
                break
            else:
                v[j], v[p] = v[p], v[j]
        else:
            if v[0] != 0:
                return v

In [13]:
def random_derangement_2(N):
    original = np.arange(N)
    new = np.random.permutation(N)
    same = np.where(original == new)[0]
    while len(same) != 0:
        swap = same[np.random.permutation(len(same))]
        new[same] = new[swap]
        same = np.where(original == new)[0]
        if len(same) == 1:
            swap = np.random.randint(0, N)
            new[[same[0], swap]] = new[[swap, same[0]]]
    return new

In [14]:
import itertools

In [15]:
def random_derangement_3(n):
    list_ex = np.arange(n)
    i = filter(lambda p: not any(i1==i2 for i1,i2 in zip(list_ex, p)), itertools.permutations(list_ex, len(list_ex)))
    return next(i)

In [16]:
%timeit -n100 random_derangement_1(2000)

6.13 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit -n900 random_derangement_2(2000)

66.3 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 900 loops each)


In [18]:
# Wont run the test...
#%timeit -n100 random_derangement_3(2000)

In [19]:
%timeit -n300 np.random.permutation(2000)

42.8 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


#### Result
random_derangement_2 is the clear winner

## Fastest random in multiple distribs

In [20]:
import scipy.stats as stats
# Middle | Ignores half for crossing over
middle = lambda n : int(n/2)
# Uniform | Okay but in nature crossing over isn't uniform
uniform = lambda n : np.random.randint(n)
# Uniform Middle | Better, but still ignores the extremis
uniformMiddle = lambda n: np.random.randint(int(n/2 - n/4), int(n/2 + n/4))
# Normal | Better, but small proba that x >= n or x < 0
normal = lambda n : int(np.random.normal(n*0.5, n*0.33))
# Normal truncated | Best solution, fits nature, but rather costly
normalTrucated = lambda n : int(stats.truncnorm(-1.5, 3-((3+n)/n), loc=n/2, scale=n*0.3).rvs(1)[0])
# Normal truncated | Best solution, fits nature, but rather costly
def normalTrucatedSingle(n):
    nrml = np.random.normal(n*0.5, n*0.33)
    if nrml < 0:
        return 0
    elif nrml >= n:
        return n-1
    return nrml
def normalTrucatedMultiple(n, size=1):
    return np.random.normal(n*0.5, n*0.33, size=size).astype(int).clip(0, n-1)

In [21]:
%timeit -n3000 middle(60000)

235 ns ± 37.7 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [22]:
%timeit -n3000 uniform(60000)

1.09 µs ± 43.9 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [23]:
%timeit -n3000 uniformMiddle(60000)

1.79 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [24]:
%timeit -n3000 normal(60000)

2.02 µs ± 454 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [25]:
%timeit -n300 normalTrucated(600)

645 µs ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


In [26]:
%timeit -n3000 normalTrucatedSingle(60000)

1.99 µs ± 753 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [27]:
%timeit -n3000 normalTrucatedMultiple(60000, 1)

4.29 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [28]:
%timeit -n3000 normalTrucatedMultiple(60000, 500)

19.1 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [29]:
%timeit -n3000 normalTrucatedMultiple(60000, 1500)

43.9 µs ± 969 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [30]:
%timeit -n3000 normalTrucatedMultiple(60000, 5000)

136 µs ± 2.76 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [31]:
t = np.array(1.0)
%timeit -n3000 t.astype(int)
%timeit -n3000 int(t)

293 ns ± 40.4 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)
145 ns ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [32]:
%timeit -n300000 123456789%2==1

26.1 ns ± 1.47 ns per loop (mean ± std. dev. of 7 runs, 300000 loops each)


In [33]:
%timeit -n300000 123456789&1==1

26.5 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 300000 loops each)


In [34]:
from scipy.optimize import rosen as rosenSciPy
rosenArray = lambda x : sum(100.0*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)
rosenArray(np.array([5,7,4,9]))

239861.0

In [35]:
rosenBivariate = lambda x, y: (1-x) ** 2 + 100*(y-x**2)**2
rosenBivariate(5,7)

32416

In [36]:
%timeit -n10000 rosenArray(np.array([1433,4521, 3432, 4324] * 90))

58.9 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
L = [1433,4521,3432,4324] * 8
%timeit -n30000 sum([rosenBivariate(L[i], L[i+1]) for i in range(len(L)-1)])

37 µs ± 661 ns per loop (mean ± std. dev. of 7 runs, 30000 loops each)


In [38]:
%timeit -n10000 rosenSciPy(np.array([1433,4521, 3432, 4324] * 90))

43.5 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
rosenBivariate(0.32, 0.879)

60.773156

In [40]:
a = np.random.rand(10)

In [44]:
size = 3000
a = np.random.random_sample((size,size))
b = np.random.random_sample((size,size))
%timeit np.dot(a,b)

355 ms ± 9.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
